# Using model options in PyBaMM
In this notebook we show how to pass options to models. This allows users to do things such as include extra physics (e.g. thermal effects) or change the macroscopic dimension of the problem (e.g. change from a 1D model to a 2+1D pouch cell model). To see all of the options currently available in PyBaMM, please take a look at the documentation [here](https://pybamm.readthedocs.io/en/latest/source/models/base_models/base_battery_model.html). For more information on combining submodels explicitly to create your own custom model, please see the [Using Submodels notebook](./using-submodels.ipynb).

## Example: Solving the SPMe with a lumped thermal model
PyBaMM is designed to be a flexible modelling package that allows users to easily include different physics within a model without having to start from scratch. In this example, we show how to pass model options to include thermal effects in the SPMe (for more information on the SPMe see [here](./models/SPMe.ipynb)). First we import PyBaMM and any other packages we need

In [1]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

We then choose out model options, which a set as a dictionary. We choose to model the behaviour in the particle using Fickian diffusion (this is the default behaviour, but we pass the option explicitly here just to demonstrate the functionality of options). We also choose a lumped thermal model (note that this is fully-coupled, i.e. parameters can depend on temperature).

In [2]:
options = {"particle": "Fickian diffusion", "thermal": "lumped"}

We then pass our options to the model 

In [3]:
model = pybamm.lithium_ion.SPMe(options)

We choose to use the parameters from [1]. We then update the heat transfer coefficients (see the [Parameter Values notebook](./parameter-values.ipynb) for more details)

In [4]:
param = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Marquis2019)
param.update(
    {
        "Negative current collector"
        + " surface heat transfer coefficient [W.m-2.K-1]": 5,
        "Positive current collector"
        + " surface heat transfer coefficient [W.m-2.K-1]": 5,
        "Negative tab heat transfer coefficient [W.m-2.K-1]": 0,
        "Positive tab heat transfer coefficient [W.m-2.K-1]": 0,
        "Edge heat transfer coefficient [W.m-2.K-1]": 0,
    }
)

We then use the default geometry, mesh and discretisation (see the [SPM notebook](./models/SPM.ipynb) for more details)

In [5]:
# create geometry
geometry = model.default_geometry

# process model and geometry
param.process_model(model)
param.process_geometry(geometry)

# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model);

We then solve using the default ODE solver for the SPMe

In [6]:
# solve model
solver = model.default_solver
t_eval = np.linspace(0, 3600, 250)
solution = solver.solve(model, t_eval)

Finally we plot the terminal voltage and the cell temperature using PyBaMM's `QuickPlot` functionality. 

In [7]:
# plot
output_variables = [
    "Terminal voltage [V]",
    "X-averaged cell temperature [K]",
    "Cell temperature [K]",
]
quick_plot = pybamm.QuickPlot(solution, output_variables)
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=3599.9999999999995, step=35.99999999999999),…

Note that the variable "X-averaged cell temperature [K]" is the scalar-valued lumped temperature, whereas the variable "Cell temperature [K]" is the value of the lumped temperature broadcasted across the whole cell domain. This type of behaviour is purposefully designed to allow easy comparison of different models and settings. For instance we may wish to compare a simulation that uses a lumped thermal model with a simulation that uses a full thermal model (i.e. one that solves the heat equation in the x-direction). When comparing these two model we could then plot the same variable "Cell temperature [K]" to compare the temperature throughout the cell. 

## References
[1] SG Marquis, V Sulzer, R Timms, CP Please and SJ Chapman. “An asymptotic derivation of a single particle model with electrolyte”. In: arXiv preprint arXiv:1905.12553 (2019).